# Matrix Factorization

Matrix factorization is a model-based collaborative filtering method that decomposes the rating matrix R into two factors, i.e. a user feature matrix and a item feature matrix.

$$\text{Ratings Matrix R} = (N \times M)$$

$$N = \text{number of users}$$

$$M = \text{number of items}$$

Recall that R is often *very large* and either can't fit into memory or will not scale as the total number of users increases.

**Problem:** We need a more efficient way to represent item-user matrix R.

**Goal:** Express matrix R using two smaller matrices *W* and *U*.

$$\hat{R} = WU^T$$

Where $\hat{R}$ is an approximation of R. We want it to be as close to R as possible, i.e. minimize error or loss.

## Matrix Factors

We want *W* and *U* to be relatively small to reduce computational complexity.

$$U^{N \times K} = \text{users matrix}$$

$$W^{M \times K} = \text{items matrix}$$

Typically use $K \in [10,50]$, but this is a hyperparameter that should be varied in order to maximize performance.

Note - there are many factors to ratings matrix R. We select a value of K and then optimize W and U in order to reduce total error, where $error = f(R, \hat{R})$.

### Example Reduction in Complexity

Let K = 10, N = 130k, M = 26k

Ratings matrix $R = N \times M = 3.38$ billion.

User matrix $U = N \times 10 = 1.3$ million.

Item matrix $W = M \times 10 = 260$k.

$U + W = 1.56 \text{million}$

**As we can see, working with matrix factors reduces computational complexity by several orders of magnitude.**

$$\frac{1.56 \text{million}}{3.38 \text{billion}} = 0.05\%$$

## Computing Ratings from U and W

Avoid multiplying *U* by *W* directly, the resulting $\hat{R}$ will be very large and consume all resources.

Instead, compute individual scalars at a time and write results to memory.

$$\hat{r}_{ij} = \hat{R}[i,j] = w_i^T u_i$$
$$w_i = W[i], \text{and } u_j = U[j]$$

## Why Does This Work

Singular value decomposition (SVD) tells us that a matrix *X* can be decomposed into 3 separate matrices multiplied together.

$$X = USV^T$$

If we can decompose a matrix into 3 matrices, then of course we can also decompose a matrix into 2 matrices. Just multiply $U \times S$.

![SVD breakdown](svd-breakdown.png)

## Interpreting Matrix Factors

We can treat the K columns of *U* and *W* as features. These features are learned during model optimization.

When we take the dot product $w_i^T u_j$, we get something similar to the cosine similarity. This tells us how well a user's attributes correlate with the item's attributes.

$$
w_i^T u_j = ||w_i|| \text{ } ||u_j|| cos\theta \propto sim(i,j)
$$

In the example below, we have K = 2 and we are *pretending* that the K features represent *comedy* and *action*.

We predict the ratings of $\hat{R}$ by taking the dot product.

![Matrix Factors Visual](matrix-factors-ex.png)

## Learned Features

When we factor R into U and W, U and W become user and item feature matrices, respectively.

**We don't actually know the real meaning of our learned K features.**

These features are latent features, and K is the latent dimensionality. These are also known as "hidden causes"

If we want to get insight on what the features represent, we can cluster users on a single feature and observe which users or items form groups.

Matrix Factorization has automatically found the optimal features for us by learning to predict $\hat{R}$ s.t. error is minimized. 

We don't have to perform manual feature engineering, and performance is usually better.

## Solving For $w_i$

$$R \approx \hat{R} = WU^T$$

How do we approximate $\hat{R}$ such that $R \approx \hat{R}$?

This is a regression problem, so we want to minimize the Sum of Squared Errors.

$$
J = \sum_{i,j \in \Omega}(r_{ij} - \hat{r}_{ij})^2
= \sum_{i,j \in \Omega}(r_{ij} - w_i^T u_j)^2
$$

This is a convex function, so we can set the gradient to 0 and solve for the parameters.

Recall:

$$\Omega = \text{the set of pairs (i,j) where user i has rated movie j}$$

$$\Psi_i = \text{the set of all movies j that user i has rated}$$

Since we are taking the derivative with respect to $w_i$, we only care about the subset of ratings that depend on $w_i$. Therefore, the summation will only care about the subset of ratingns that depend on user *i* has rated. 

$$
\frac{\partial J}{\partial w_i} =
2 \sum_{j \in \Psi_i}(r_{ij} - w_i^T u_j)(-u_j) = 0
$$

Dot products are commutative, so we can change order. Therefore:

$$
\sum_{j \in \Psi_i}r_{ij}u_j = \sum_{j \in \Psi_i}(u_j^T w_i)u_j
 = \sum_{j \in \Psi_i}u_j u_j^T w_i 
$$

Since this is a summation over j and $w_i$ does not depend on j.

$$
\left( \sum_{j \in \Psi_i}u_j u_j^T \right) w_i =
\sum_{j \in \Psi_i}r_{ij}u_j
$$

Now we are left with Ax = b, and we can solve for x.

$$
w_i = \left( \sum_{j \in \Psi_i}u_j u_j^T \right)^{-1} \sum_{j \in \Psi_i}r_{ij}u_j
$$

```python
x = np.linalg.solve(A, b)
```

## Solving For $u_i$

The loss is symmetric in W and U, so steps are identical.

$$
\frac{\partial J}{\partial u_j} =
2 \sum_{i \in \Omega_j}(r_{ij} - w_i^T u_j)(-w_i) = 0
$$

Set the derivative to 0, isolate terms, and put into form Ax = b:

$$
u_j = \left( \sum_{i \in \Omega_j}w_i w_i^T \right)^{-1}
\sum_{i \in \Omega_j}r_{ij}w_i
$$

## 2-Way Dependency Between W and U

Solving for W depennds on U, and solving for U depends on W.

### Alternating Least Squares

Initialize W and U to random numbers.

```python
W = randn(N, K)
U = randn(M, K)
```

Then apply the two equations iteratively, solving for W and then U.

$$
w_i = \left( \sum_{j \in \Psi_i}u_j u_j^T \right)^{-1} \sum_{j \in \Psi_i}r_{ij}u_j
$$

$$
u_j = \left( \sum_{i \in \Omega_j}w_i w_i^T \right)^{-1}
\sum_{i \in \Omega_j}r_{ij}w_i
$$

Each iteration will take 1 step towards the global minimum.

It does not matter which matrix we solve for first, because W and U are symmetric.

Let's say you update W first, and then U. You do not need to use *old* values of W when solving for U. It is more efficient to use the new values for *W*.

## Incorporating User Bias

Recall our intro to User-Based Collaborative Filtering, we used rating deviations instead of raw rating. We achieved this by substracting a user's average rating from each item rating. We did this because each user has a bias and may be optimistic/pessimistic with respect to ratings.

We should incorporate a similar bias into our Matrix Factorization.

$$
\hat{r}_{ij} = w_i^T u_j + b_i + c_j + u
$$

$$
b_i = \text{ user bias}
$$

$$
c_j = \text{ item bias}
$$

$$
u = \text{ global average}
$$

The global average $u$ centers the data set, i.e. shifting ratings so that the mean is 0.

User bias is discussed in User-User CF - users may be optimistic or pessimistic in their ratings.

Item bias is important because two movies can have the same features, e.g. action, comedy, etc., but one can be terribly rated compared to the other.

### Training with Bias Terms

$$J = \sum_{i,j \in \Omega}(r_{ij} - \hat{r}_{ij})^2$$

$$\hat{r}_{ij} = w_i^T u_j + b_i + c_j + u$$

#### Differentiat wrt $w_i$

$$
\frac{\partial J}{\partial w_i} =
2 \sum_{j \in \Psi_i}(r_{ij} - w_i^T u_j - b_i - c_j - u)(-u_j) = 0
$$

$$
\sum_{j \in \Psi_i}(w_i^T u_j)u_j = 
\sum_{j \in \Psi_i}(r_{ij} - b_i - c_j - u)u_j
$$

$$
w_i = \left(\sum_{j \in \Psi_i}u_j u_j^T\right)^{-1}
\sum_{j \in \Psi_i}(r_{ij} - b_i - c_j - u)u_j
$$

#### Differentiating wrt $u_j$

$$
u_j = \left(\sum_{i \in \Omega_j}w_i w_i^T\right)^{-1}
\sum_{i \in \Omega_j}(r_{ij} - b_i - c_j - u)w_i
$$

#### Differentiating wrt $b_i$

Since $b_i$ appears by itself its derivative is 1 and all other terms go to 0.

$$
\frac{\partial J}{\partial b_i} =
2 \sum_{j \in \Psi_i}(r_{ij} - w_i^T u_j - b_i - c_j - u)(-1) = 0
$$

$$
b_i = \frac{1}{\mid \Psi_i \mid} \sum_{j \in \Psi_i}(r_{ij} - w_i^T u_j - c_j - u)
$$

The Bias for user i is the average deviation of the actual rating minus every other part of the model.

#### Differentiating wrt $c_j$

Similar to $b_i$.

$$
c_j = \frac{1}{\mid \Psi_j \mid} \sum_{i \in \Psi_j}(r_{ij} - w_i^T u_j - b_i - u)
$$

#### Computing $u$

$u$ is a global average and does not need to be updated. Just take it from the training data prior to training.